In [ ]:
import os
from dotenv import load_dotenv
import openai
import json
import boto3
import findspark
from pyspark.sql import *
from pyspark import SparkConf, SparkContext
from py4j.java_gateway import java_import
findspark.init()
from pyspark.sql import SparkSession
import csv
import io
from urllib.parse import urlparse

import pandasai as pai
from pandasai_openai import OpenAI
import pandas as pd
import snowflake.connector



In [ ]:
!rm /Users/satyak/iceberg/milvus/class/cache/cache_db_0.12.db

In [ ]:
full_path_to_warehouse = "/tmp/ozone1/warehouse"
hive_warehouse ="/tmp/ozone1/warehouse"

In [ ]:
load_dotenv()
openai_api_key=os.getenv("OPENAI_API_KEY")

llm = OpenAI(api_token=openai_api_key)
pai.config.set({"llm": llm,})

In [ ]:
spark = SparkSession.builder \
        .config('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.4,org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.8.0,software.amazon.awssdk:bundle:2.17.178,software.amazon.awssdk:url-connection-client:2.17.178')\
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
        .config("spark.sql.catalog.spark_catalog","org.apache.iceberg.spark.SparkSessionCatalog") \
        .config("spark.sql.catalog.spark_catalog.type","hadoop") \
        .config("spark.sql.catalog.spark_catalog.warehouse",hive_warehouse) \
        .config("spark.sql.warehouse.dir",hive_warehouse) \
        .config("spark.sql.catalog.iceberg_catalog","org.apache.iceberg.spark.SparkCatalog") \
        .config("spark.sql.catalog.iceberg_catalog.type","hadoop") \
        .config("spark.sql.catalog.iceberg_catalog.warehouse",hive_warehouse) \
        .getOrCreate()

In [ ]:
score=spark.sql(f"""select * from  iceberg_catalog.prod.exam_scores """)

In [ ]:
score.show()

In [ ]:
score_pandas_df = score.toPandas()

In [ ]:
names=spark.sql(f"""select * from  iceberg_catalog.prod.students """)
names.show()

In [ ]:
names_pandas_df = names.toPandas()

### Convert pd to pai datafram

In [ ]:
score_pai = pai.DataFrame(score_pandas_df)
names_pai = pai.DataFrame(names_pandas_df)


In [ ]:
pai.chat("How many rows are there",score_pai,names_pai)

In [ ]:
pai.chat("show who  has top score. Show in row format. Don't show graph",score_pai,names_pai)

In [ ]:
pai.chat("show who  has top score. Show in row format. Don't show graph",score_pai)